In [1]:
import requests
import pandas as pd
import datetime
import time
import matplotlib.pyplot as plt

In [10]:
# Tu clave de API de Alpha Vantage
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo
API_KEY = "V7K1X8BPXINPXFOV"
#API_KEY = "demo"

# Función para descargar datos de Alpha Vantage
def download_stock_data(symbol, start, end):
    url = f"https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": symbol,
        "outputsize": "full",
        "apikey": API_KEY
    }
    
    print(f"Descargando datos de {symbol} desde Alpha Vantage...")
    r = requests.get(url, params=params)
    data = r.json()

    if "Time Series (Daily)" not in data:
        print("Error en la descarga:", data)
        return None
    
    df = pd.DataFrame.from_dict(data["Time Series (Daily)"], orient="index")
    df = df.rename(columns={
        '1. open': 'Open',
        '2. high': 'High',
        '3. low': 'Low',
        '4. close': 'Close',
        '5. adjusted close': 'Adj Close',
        '6. volume': 'Volume'
    })
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    df = df.loc[start:end]
    df = df.astype(float)
    return df

# Función ficticia para análisis Vegas Channel (reemplaza con tu lógica real)
def vegas_channel_analysis(df):
    df['EMA12'] = df['Adj Close'].ewm(span=12).mean()
    df['EMA26'] = df['Adj Close'].ewm(span=26).mean()
    return df

# Función para graficar
def plot_vegas_channel(df, symbol, target_date=None, target_price=None):
    plt.figure(figsize=(12,6))
    plt.plot(df.index, df['Adj Close'], label='Precio ajustado')
    plt.plot(df.index, df['EMA12'], label='EMA 12')
    plt.plot(df.index, df['EMA26'], label='EMA 26')
    if target_date and target_price:
        plt.axvline(pd.to_datetime(target_date), color='gray', linestyle='--')
        plt.axhline(target_price, color='gray', linestyle='--')
        plt.scatter(pd.to_datetime(target_date), target_price, color='red')
    plt.title(f'Vegas Channel Analysis: {symbol}')
    plt.xlabel('Fecha')
    plt.ylabel('Precio ($USD)')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [11]:
# Fechas
symbol = 'TSLA'
today = datetime.date.today()
w_days = 7
n_weeks = 51
start_date = today - datetime.timedelta(days=w_days * n_weeks)
end_date = today

# Ejecutar
df = download_stock_data(symbol, start=start_date, end=end_date)
if df is not None:
    df = vegas_channel_analysis(df)
    plot_vegas_channel(df, symbol, '2024-12-26', 439.59)

Descargando datos de TSLA desde Alpha Vantage...
Error en la descarga: {'Information': 'Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints'}
